In [1]:
import pandas as pd
import numpy as np
import pyodbc 
import time
from datetime import datetime
from datetime import date
import sqlite3
from sqlalchemy import create_engine

In [3]:
import logging
path = r'..\..\Logs\clinical_log.log'
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("Determine Discharge Status")
# logger.info("testing log")


In [3]:
def findPatientsNotSeenInaWhile(lastApptDate):
    try:
        if (datetime.now() - lastApptDate).days > 21:
            if (datetime.now() - lastApptDate).days > 91:
                if (datetime.now() - lastApptDate).days > 183:
                    return "Hasn't been seen in 6 months" 
                else:
                    return "Hasn't been seen in 3 months"
            else:
                return "Hasn't been seen in 3 weeks"
        else:
            return "They've been seen in the past 3 weeks" 
    except:        
        return "Never Seen"   

In [7]:
# diff path from TestTev vs WorkingEnv
# conn = sqlite3.connect('../../InSyncConnection/Database/InSyncClinical.db')
conn = create_engine(r'mssql+pyodbc://@PYTHONSERVER\SQLEXPRESS/InSync?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes', fast_executemany=True)
# cursor = conn.cursor()

# printing all table names  
# sql_query = """SELECT name FROM sqlite_master
#     WHERE type='table';"""

# cursor.execute(sql_query)
# print(cursor.fetchall())

# Read in Data

In [5]:
testPatients = '''
SELECT
    MRNNumber
FROM 
    emr_PatientDetails
WHERE 
    LOWER(FirstName) LIKE '%test%'
    OR LOWER(FirstName) LIKE '%patient%'
    OR LOWER(FirstName) LIKE '%new%'
    OR LOWER(FirstName) LIKE '%client%'
    OR LOWER(FirstName) LIKE '%release%'
    OR LOWER(LastName) LIKE '%test%'
    OR LOWER(LastName) LIKE '%patient%'
    OR LOWER(LastName) LIKE '%fake%'
    OR CAST(MRNNumber AS INTEGER) < 53
    OR CAST(MRNNumber AS INTEGER) = 54
    OR CAST(MRNNumber AS INTEGER) = 141
    OR CAST(MRNNumber AS INTEGER) = 4155
    OR CAST(MRNNumber AS INTEGER) = 4154
    OR CAST(MRNNumber AS INTEGER) = 7170
'''

In [8]:
sql=f'''
SELECT 
    tblEncounterSummary.PatientID as PatientID,
    emr_PatientDetails.isActive as IsActive,
    tblEncounterSummary.VisitDateTime as EncounterDate,
    tblEncounterType.EncounterType
FROM 
    tblEncounterSummary
    LEFT JOIN emr_PatientDetails ON (tblEncounterSummary.patientID = emr_PatientDetails.patientID)
    LEFT JOIN tblEncounterType ON (tblEncounterSummary.EncounterTypeID=tblEncounterType.EncounterTypeID)
WHERE
    emr_PatientDetails.MRNNumber NOT IN ({testPatients})
ORDER BY EncounterDate
'''
try:
    encounters = pd.read_sql(sql, conn)
    logger.info("Successfully queried encounters")
except Exception as e:
    logger.error(f"Failed to query encounters.", exc_info=True) 
    print(e)

encounters

,PatientID,IsActive,EncounterDate,EncounterType
0,622786,1,2002-10-16 12:39:00,Admission Note
1,622241,1,2016-02-01 11:31:00,Initial Treatment Plan
2,622220,1,2016-12-27 15:29:00,Initial Treatment Plan
3,620580,1,2019-03-28 16:47:00,Initial Treatment Plan
4,622453,1,2019-07-10 18:43:00,Initial Evaluation-1
...,...,...,...,...
287560,620991,1,2024-08-28 16:00:00,Treatment Plan Review
287561,625740,1,2024-08-29 16:45:00,Treatment Plan Review
287562,622094,1,2024-09-07 22:00:00,Treatment Plan Review
287563,628157,1,2024-09-13 16:37:00,CFTSS Treatment Plan Review


In [9]:
sql=f'''
SELECT 
    tblEncounterSummary.PatientID as PatientID,
    tblEncounterSummary.VisitDateTime as Billable_EncounterDate,
    tblEncounterType.EncounterType as Billable_EncounterType,
    tblEncounterSummary.ProgramName,
    tblFacilitiesMaster.FacilityName
FROM 
    tblEncounterSummary
    LEFT JOIN emr_PatientDetails ON (tblEncounterSummary.patientID = emr_PatientDetails.patientID)
    LEFT JOIN tblEncounterType ON (tblEncounterSummary.EncounterTypeID=tblEncounterType.EncounterTypeID)
    LEFT JOIN tblFacilitiesMaster ON (tblEncounterSummary.FacilityId=tblFacilitiesMaster.FacilityId)
WHERE IsBillable LIKE '%TRUE%' 
    AND emr_PatientDetails.MRNNumber NOT IN ({testPatients})
Order By Billable_EncounterDate    
'''
try:
    billableEncounters = pd.read_sql(sql, conn)
    logger.info("Successfully queried billableEncounters = pd.read_sql(sql, conn)")
except Exception as e:
    logger.error(f"Failed to query billableEncounters = pd.read_sql(sql, conn).", exc_info=True) 
    print(e)


# Transform Data

In [10]:
# get the most recent (last) encounter type, date and patient activity
discharged = encounters.groupby(['PatientID'], as_index=False)[['EncounterType', 'EncounterDate', 'IsActive']].last()
finalBillable = billableEncounters.groupby(['PatientID'], as_index=False)[['Billable_EncounterType', 'Billable_EncounterDate']].last()
finalEncounterFacility = billableEncounters.groupby(['PatientID'], as_index=False)[['ProgramName', 'FacilityName']].last()

In [11]:
# check if encounter type contains discharge
discharged['is_Discharged'] = discharged['EncounterType'].map(lambda type: 
                                                              1 if "discharge" in str(type).lower() else 
                                                              (1 if "non-admission note" in str(type).lower() else
                                                              (1 if "closed case summery" in str(type).lower() else 0)))
# summarize last seen date
finalBillable['seen_Recently'] = finalBillable['Billable_EncounterDate'].map(findPatientsNotSeenInaWhile)

In [12]:
finalPatientStatus = discharged.merge(finalBillable, how='left', on='PatientID').merge(finalEncounterFacility, how='left', on='PatientID')

In [13]:
# rename columns to be more explicit
finalPatientStatus.columns = ['PatientID', 
                      'finalEncounterType', 
                      'finalEncounterDate', 
                      'is_ActiveInInSync',
                      'is_lastEncounterDischarge',
                      'lastBillable_EncounterType',
                      'lastBillable_EncounterDate',
                      'seen_Recently',
                      'lastBillable_ProgramName',
                      'lastBillable_FacilityName']
#reorder the columns
finalPatientStatus =    finalPatientStatus[['PatientID', 
                      'finalEncounterType', 
                      'finalEncounterDate', 
                      'is_ActiveInInSync',
                      'is_lastEncounterDischarge',
                      'lastBillable_EncounterType',
                      'lastBillable_EncounterDate',
                      'lastBillable_ProgramName',
                      'lastBillable_FacilityName',
                      'seen_Recently']       ]

In [14]:
finalPatientStatus[finalPatientStatus['PatientID'] == 622719]

,PatientID,finalEncounterType,finalEncounterDate,is_ActiveInInSync,is_lastEncounterDischarge,lastBillable_EncounterType,lastBillable_EncounterDate,lastBillable_ProgramName,lastBillable_FacilityName,seen_Recently
1847,622719,Psychotherapy 45 Minutes,2023-05-31 14:30:00,1,0,Psychotherapy 45 Minutes,2023-05-31 14:30:00,Boro Park Clinics,Clinic 5309 18th Avenue,Hasn't been seen in 6 months


## Manual Line
- The code below is used to help with writing the proper SQL Create table query below

In [15]:
# use to transform data into
", ".join([item + " " + str(finalPatientStatus[item].dtype) for item in finalPatientStatus.columns])

'PatientID int64, finalEncounterType object, finalEncounterDate datetime64[ns], is_ActiveInInSync int64, is_lastEncounterDischarge int64, lastBillable_EncounterType object, lastBillable_EncounterDate datetime64[ns], lastBillable_ProgramName object, lastBillable_FacilityName object, seen_Recently object'

# Push Data to Table

In [16]:
table_name= "ptPatient_Activity"
try:
    # conn.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER, finalEncounterType TEXT, finalEncounterDate TEXT, is_ActiveInInSync INTEGER, is_lastEncounterDischarge INTEGER, lastBillable_EncounterType TEXT, lastBillable_EncounterDate TEXT, lastBillable_ProgramName TEXT, lastBillable_FacilityName TEXT, seen_Recently TEXT) ''')
    # conn.commit()
    finalPatientStatus.to_sql("ptPatient_Activity", conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed {table_name} to database.")            
except Exception as e:
    logger.error(f"Failed to push {table_name} to database.", exc_info=True) 
    print(e)

In [17]:
conn.dispose()